In [8]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
input_str = '<' + input_str + '>'
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
           'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', '\'', '>']
print(f'{letters=}')
# print(f'{input_str=}')

len(letters)=32
502
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', "'", '>']


In [10]:
# generate SDR

out_len = 128 # 128
sparsity = 4 # 4
max_overlap = 1 # 1

sparse_prob = sparsity / out_len

letter_SDRs = {}

for letter in letters:
    while True:
        rnd_SDR = np.random.choice([0, 1], out_len, True, [1 - sparse_prob, sparse_prob])

        if (rnd_SDR > 0).sum() != sparsity:
            continue
        
        is_over_max = False
        for SDR in letter_SDRs.values():
            if (SDR & rnd_SDR).sum() > max_overlap:
                is_over_max = True
                break
        
        if is_over_max:
            continue
            
        break
    
    # print(f'{letter}')
    letter_SDRs.setdefault(letter, rnd_SDR)

for letter, SDR in letter_SDRs.items():
    print(f'{letter} : {(SDR > 0).sum()} {SDR.tolist()}')

a : 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
b : 4 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c : 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0

In [11]:
# text to SDRs
input_SDRs = []

for inp in input_str:
    input_SDRs.append(letter_SDRs[inp])

print(f'{len(input_SDRs)=}')

# for SDR in input_SDRs:
#     print(f'{SDR.tolist()}')

len(input_SDRs)=502


In [12]:
# init start column
column_size = 16

start_SDR = letter_SDRs['<']

start_I = np.zeros((start_SDR.size, column_size))

active_start_SDR_idx = np.where(start_SDR == 1)

for as_idx in active_start_SDR_idx[0]:
    rnd_active = np.random.randint(column_size)

    start_I[as_idx][rnd_active] = 1

# for col in np.rot90(start_I):
#     print(f'{col.astype(int).tolist()}')

In [18]:
from numpy import ndarray


def train(input_SDRs: list[ndarray], push_delta=0.04, epoch_cnt=1, inp_len=128, out_len=128, sparsity=4):
    I = np.zeros(inp_len)
    K = np.zeros(out_len)

    S = np.random.rand(out_len, out_len) * 1e-2

    V = np.zeros(out_len)

    prec = 1e-30

    eps0 = 0.02

    eps0_V = 0.09

    inputs_idx = np.arange(len(input_SDRs))

    for epoch_idx in range(epoch_cnt):
        np.random.shuffle(inputs_idx)

        eps = (epoch_cnt - epoch_idx) * eps0

        for SDR_idx in inputs_idx:
            I = np.copy(input_SDRs[SDR_idx])

            K = np.dot(S, I)

            K /= np.max(np.abs(K)) + prec

            nV = np.abs(V - np.max(V))
            nV /= np.max(nV) + prec

            OV = K + nV * eps0_V

            max_O_idx = np.argsort(OV)[::-1][:sparsity]
            pull_idx = max_O_idx[0]
            push_idx = max_O_idx[1:sparsity]

            S[pull_idx] += (I - S[pull_idx] * K[pull_idx][np.newaxis].T) * eps
            S[push_idx] += (I - S[push_idx] * K[push_idx][np.newaxis].T) * (-push_delta) * eps

            # winners
            V[pull_idx] += 1

    print(np.sum(V > 0).tolist())
    return S


S = train(input_SDRs=input_SDRs, epoch_cnt=100, out_len=128, sparsity=4)
print(f'{S.max()=} {S.min()=} {np.mean(S)=}')


84
S.max()=1.0771685743999553 S.min()=-1.6524250939475371 np.mean(S)=-0.09758803870362051


In [6]:
I = np.zeros((out_len, column_size))
K = np.zeros((out_len, column_size))

S = np.random.rand(out_len, column_size, out_len * column_size) * 1e-2

V = np.zeros((out_len, column_size))

prec = 1e-30

# train

epoch_cnt = 1

eps0 = 0.003

eps0_V = 0.09

for epoch_idx in range(epoch_cnt):
    I = np.copy(start_I)

    for SDR in input_SDRs[1:]:
        I = I.reshape(out_len * column_size)

        K = np.dot(S, I)

        K /= np.max(np.abs(K)) + prec

        nV = np.abs(V - np.max(V, axis=1)[np.newaxis].T)
        nV /= np.max(nV, axis=1)[np.newaxis].T + prec

        OV = K + nV * eps0_V

        OV *= SDR[np.newaxis].T
        max_O = np.max(OV, axis=1)

        K *= SDR[np.newaxis].T
        K[OV < max_O[np.newaxis].T] = 0
        max_O_idx = np.where(K != 0)

        S[max_O_idx] += (I - S[max_O_idx] * K[max_O_idx][..., np.newaxis]) * eps0

        # winners
        V[max_O_idx] += 1

        # recur
        K[K != 0] = 1

        I = np.copy(K)

# O.shape
print(np.mean(V, axis=1).tolist())
print(np.sum(V == 0))
print(np.sum((V > 0), axis=1).tolist())
# for col in np.rot90(V):
#     print(f'{col.astype(int).tolist()}')

[0.125, 0.0, 0.0, 0.96875, 1.28125, 0.40625, 0.0, 1.71875, 0.0, 1.09375, 1.21875, 0.71875, 0.03125, 0.0, 1.09375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03125, 0.0, 0.28125, 0.0, 0.0, 0.40625, 0.0, 1.0, 3.34375, 0.0, 1.09375, 0.0, 0.25, 1.09375, 0.0, 0.0, 0.0, 0.90625, 1.5625, 0.0, 0.0, 1.0, 0.40625, 0.75, 5.65625, 0.8125, 0.0, 0.0, 0.0, 1.03125, 0.03125, 0.375, 0.0, 1.8125, 0.0, 0.0, 0.0, 0.75, 0.0, 0.84375, 0.25, 1.09375, 1.125, 0.375, 0.03125, 0.0, 0.0, 0.84375, 0.0, 1.0, 0.0, 0.0, 1.46875, 0.40625, 0.0, 1.59375, 0.0, 0.0, 0.0, 0.28125, 1.09375, 0.0, 0.0, 1.125, 0.28125, 0.0, 1.03125, 0.0, 0.0, 3.28125, 0.0, 0.65625, 0.75, 1.09375, 0.0, 0.03125, 0.90625, 0.53125, 0.0, 0.34375, 0.0, 0.6875, 0.09375, 0.09375, 0.21875, 3.3125, 0.28125, 1.5625, 0.21875, 0.53125, 0.0, 0.90625, 0.125, 0.0, 0.0, 0.0, 0.0, 0.625, 1.03125, 0.0, 0.0, 0.25, 0.40625, 0.0, 0.34375, 0.25, 0.0]
2863
[4, 0, 0, 24, 28, 11, 0, 30, 0, 24, 24, 20, 1, 0, 26, 0, 0, 0, 0, 0, 0, 1, 0, 9, 0, 0, 13, 0, 25, 32, 0, 27, 0, 8, 27, 0, 0

In [7]:
# test
error = []

I = np.copy(start_I)

for SDR in input_SDRs[1:]:
    I = I.reshape(out_len * column_size)

    K = np.dot(S, I)

    max_O = np.max(K, axis=1)
    argmax_O_idx = np.argsort(max_O)[::-1][:sparsity]

    SDR_O = np.zeros(K.shape[0])
    SDR_O[argmax_O_idx] = 1

    err = (SDR.astype(int) ^ SDR_O.astype(int)).sum()
    # print(f'{err=}')
    error.append(err)

    K *= SDR[np.newaxis].T
    max_O = np.max(K, axis=1)
    K[K < max_O[np.newaxis].T] = 0

    # recur
    K[K != 0] = 1

    I = np.copy(K)

error = np.array(error)
mean_percent = int(np.mean(error) / (2 * sparsity) * 100)
print(f'{np.mean(error)=} {error.max()=} {error.min()=} {mean_percent=}')

# O.shape
# for col in np.rot90(O):
#     print(f'{col.astype(int).tolist()}')

np.mean(error)=0.15568862275449102 error.max()=6 error.min()=0 mean_percent=1
